In [ ]:
import numpy as np
from numpy.polynomial import Polynomial as P
from numpy.polynomial import chebyshev as c
from numpy.polynomial import Chebyshev as C
from numpy.polynomial import Legendre as L
from math import factorial as fac

In [5]:
def is_psd_for_sym(A):
    m , n = A.shape
    if m == 1 or n == 1:
        return True,'0'
    # Determine if it's symmetry

    # Case 1
    if A[0,0] < 0:
        return False,'1'
    
    # Case 2
    elif A[0,0] == 0:
        for j in range(n):
            if A[0,j] != 0:
                return False,'2'
        return is_psd_for_sym(A[1: , 1:])[0],'2'
    
    # Case 3
    elif A[0,0] > 0:
        B = A.copy()
        for i in range(1,m):
            B[i] -= A[i,0] / A[0,0] * A[1]
        return is_psd_for_sym(B[1:,1:])[0],'3'
    
def is_psd_eig(M, atol=1e-12):
    """
    Returns True if all eigenvalues of M are >= -atol.
    atol is a numerical tolerance to allow for floating-point inaccuracies.
    """
    w, _ = np.linalg.eigh(M)  # Eigenvalues in ascending order
    return np.all(w >= -atol)    
    


In [11]:
import numpy as np
import scipy.linalg
import scipy.special

def chebyshev_moments_from_monomial(moments, degree):
    """
    Convert monomial moments to Chebyshev moments using recurrence relations.
    moments: Array of monomial moments m_k = ∫ x^k w(x) dx
    degree: Maximum degree of Chebyshev moments required
    Returns:
        chebyshev_moments: Transformed moments in the Chebyshev basis
    """
    N = degree
    c_moments = np.zeros(N+1)
    
    # Compute Chebyshev moments from monomial moments using transformation formulas
    for k in range(N+1):
        for j in range(k+1):
            if (k-j) % 2 == 0:  # Only even terms contribute
                c_moments[k] += (2 if j > 0 else 1) * moments[j] * scipy.special.binom(k, j) / (2 ** k)

    return c_moments

def build_chebyshev_hankel_matrix(chebyshev_moments, r):
    """
    Constructs the Hankel matrix in the Chebyshev basis.
    chebyshev_moments: Array of Chebyshev moments
    r: Degree of polynomial
    """
    H = np.array([[chebyshev_moments[i+j] for j in range(r)] for i in range(r)])
    return H

def solve_chebyshev_quadrature(chebyshev_moments, r):
    """
    Solve for the coefficients of the orthogonal polynomial in the Chebyshev basis.
    """
    H = build_chebyshev_hankel_matrix(chebyshev_moments, r)
    b = -chebyshev_moments[r:r+r]  # Right-hand side vector
    alpha = np.linalg.solve(H, b)
    
    # Construct polynomial in Chebyshev basis
    chebyshev_poly_coeffs = np.concatenate(([1], alpha))  # Leading coefficient is 1
    return chebyshev_poly_coeffs

def find_quadrature_nodes(chebyshev_poly_coeffs):
    """
    Compute the roots of the Chebyshev polynomial to obtain the quadrature nodes.
    """
    roots = np.polynomial.chebyshev.chebroots(chebyshev_poly_coeffs)
    return roots
r = 4  # Degree of quadrature polynomial
# Example: Define Monomial Moments (Normally Computed from Weight Function)
monomial_moments = moments = np.array([(1**(i+1) - (-1)**(i+1)) / (i+1) for i in range(2 * r+1)])


# Convert to Chebyshev Moments   chebyshev_moments_from_monomial(monomial_moments, 2*r)

chebyshev_moments = np.array([inte_p(C.basis(i) , -1 , 1) for i in range(2 * r)])

# Solve for the Quadrature Polynomial
chebyshev_poly_coeffs = solve_chebyshev_quadrature(chebyshev_moments, r)

# Find Quadrature Nodes
quadrature_nodes = find_quadrature_nodes(chebyshev_poly_coeffs)

# Print Results
print("Chebyshev Moments:", chebyshev_moments)
print("Quadrature Polynomial Coefficients (Chebyshev Basis):", chebyshev_poly_coeffs)
print("Quadrature Nodes:", quadrature_nodes)


Chebyshev Moments: [ 2.          0.         -0.66666667  0.         -0.13333333  0.
 -0.05714286  0.        ]
Quadrature Polynomial Coefficients (Chebyshev Basis): [ 1.         -0.02857143 -0.         -0.28571429 -0.        ]
Quadrature Nodes: [-0.60235329-0.60290004j -0.60235329+0.60290004j  1.20470658+0.j        ]


In [13]:
import numpy as np
import numpy.polynomial.chebyshev as cheb

def chebyshev_poly_roots(a_coeffs):
    """
    Find roots of the monic polynomial:
        P(x) = x^r - sum(a_i * T_i(x))
    
    Parameters:
    a_coeffs (array-like): Coefficients [a_0, a_1, ..., a_{r-1}] in Chebyshev basis.
    
    Returns:
    roots (array): The computed roots of P(x).
    """
    r = len(a_coeffs)  # Degree of the polynomial

    # Define the polynomial in Chebyshev basis: x^r - sum a_i * T_i(x)
    chebyshev_coeffs = np.zeros(r + 1)
    chebyshev_coeffs[r] = 1  # Leading coefficient for x^r
    chebyshev_coeffs[:r] -= a_coeffs  # Subtracting the given Chebyshev coefficients

    # Find the roots of the polynomial in Chebyshev basis\
    print(cheb.Chebyshev(chebyshev_coeffs))
    roots = cheb.Chebyshev(chebyshev_coeffs).roots()
    return roots

# Example usage
a_coeffs = [0.5, -0.3, 0.1, -0.05]  # Coefficients for T_0, T_1, ..., T_{r-1}
roots = chebyshev_poly_roots(a_coeffs)
print("Roots of the polynomial:", roots)


-0.5 + 0.3 T_1(x) - 0.1 T_2(x) + 0.05 T_3(x) + 1.0 T_4(x)
Roots of the polynomial: [-0.99660261 -0.27007481  0.29404642  0.94763101]
